### Initialize

In [1]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.debias import debias
import vector_math_jb as vm 

# Standard python helper libraries.
import os, sys, re, json, time
import itertools, collections
#from importlib import reload
from IPython.display import display

# NumPy and SciPy for matrix ops
import numpy as np
import scipy.sparse

# Visualize
import matplotlib.pyplot as plt

# NLTK for NLP utils
import nltk

# PCA 
from sklearn.decomposition import PCA

# Helper libraries
from w266_common import utils, vocabulary#, tf_embed_viz

## Load embeddings

In [60]:
# load subset of word embedding trained on Google News text
E_gn = WordEmbedding("./embeddings/w2v_gnews_small.txt")
E_jp = WordEmbedding("./embeddings/RANE_300d_english_50k.txt")
E_gn_db = WordEmbedding("./embeddings/w2v_gnews_small_debiased.txt")

*** Reading data from ./embeddings/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
*** Reading data from ./embeddings/RANE_300d_english_50k.txt
(50000, 300)
50000 words of dimension 300 : the, to, of, and, ..., prin, synchronise, dissertations, mammography
50000 words of dimension 300 : the, to, of, and, ..., prin, synchronise, dissertations, mammography
*** Reading data from ./embeddings/w2v_gnews_small_debiased.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine


### 1. Word Similarity Evaluation

In [74]:
# adapted from Faruqui and Dyer, Community Evaluation and Exchange of Word Vectors at wordvectors.org
# RG = Rubenstein and Goodenough, 1965; WS = Finkelstein et. al, 2002 -- benchmarks used in Bolukbasi et al.

%run 'evaluation/all_wordsim_jb.py' 'embeddings/' 'evaluation/data/word-sim/'

File #                     Embeddings      RG (53/65)    WS (318/353)
     1                  article_embed          0.5378          0.5702 

     2          RANE_300d_english_50k          0.6052          0.6159 

     3                w2v_gnews_small          0.7618          0.6857 

     4       w2v_gnews_small_debiased          0.7601          0.6826 



### 3. MSR Analogies Eval

In [3]:
with open('./evaluation/data/analogies.json', "r") as f:
    analogies = json.load(f) # This is HUGE

In [48]:
with open('analogies_shorter.json', "r") as f:
    analogies_shorter = json.load(f) 

In [55]:
with open('./evaluation/data/analogies_2000.json', "r") as f:
    analogies_final = json.load(f) 

In [57]:
def compare_analogies(analogy_list,e):
    num_analogies = 0
    correct = 0  
    num_missing = 0
    #keep_analogies = []
    t0 = time.time()
    print_freq = 500

    for i,wds in enumerate(analogy_list):
        # we'll get KeyError if a word can't be found
        #print(wds)
        try:
            
            # generate "d" in a:b::c:d analogies, given a, b, and c
            d = vm.show_analogy(e,wds[0],wds[1],wds[2],5)[0] # needs to be at least 2 

            # keep track of how many analogies were computed
            num_analogies += 1
            #keep_analogies.append(wds)

            if d == wds[3]:
                correct += 1
                
        except:
            #bad_analogies.append(wds)
            num_missing += 1
        
        if i % print_freq  == 0:
            print("Completed {:d} analogies in {:s}".format(i, utils.pretty_timedelta(since=t0)))
    
    try: # if we're running on a subset, we may actually not be able to do _any_ analogies
        score = correct / num_analogies
    except:
        score = 0
                
    print("Computed {:d}/{:d} analogies correctly in {:s}, accuracy: {:.2f}".format(correct,num_analogies,utils.pretty_timedelta(since=t0),score))
    
    #return keep_analogies

In [59]:
compare_analogies(analogies_final,E_gn) 

Completed 0 analogies in 0:00:00
Completed 500 analogies in 0:03:58
Completed 1000 analogies in 0:08:48
Completed 1500 analogies in 0:23:30
Computed 1379/2000 analogies correctly in 0:28:34, accuracy: 0.69


In [63]:
compare_analogies(analogies_final[:100],E_gn)

Completed 0 analogies in 0:00:00
Computed 62/100 analogies correctly in 0:00:47, accuracy: 0.62


In [64]:
compare_analogies(analogies_final[:100],E_gn_db)

Completed 0 analogies in 0:00:00
Computed 61/100 analogies correctly in 0:00:48, accuracy: 0.61


In [37]:
def intersect(l1, l2):
    return [wds for wds in l1 if wds in l2]

In [26]:
def remove(l1,l2):
    for i,item in enumerate(l1):
        l2.remove(item)

In [46]:
with open('analogies_shorter.json', "w") as f:
    json.dump(short_copy,f) 

In [54]:
with open('analogies_2000.json', "w") as f:
    json.dump(list_of_random_items,f) 

In [53]:
## Select just 2000 random analogies
import random

list_of_random_items = random.sample(analogies_shorter, 2000)
